# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = 'data/song_data'

In [5]:
filepath = get_files(song_files)

In [6]:
df1 = pd.read_json(filepath[0], lines = True)
#df2 = pd.read_json(filepath[5], lines = True)
df_song = pd.concat([df1])
df_song.head()

,artist_id,artist_latitude,artist_location,artist_longitude,artist_name,duration,num_songs,song_id,title,year
0,ARNF6401187FB57032,40.79086,"New York, NY [Manhattan]",-73.96644,Sophie B. Hawkins,305.162,1,SONWXQJ12A8C134D94,The Ballad Of Sleeping Beauty,1994


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
song_cols = ["song_id", "title", "artist_id", "year", "duration"]
song_data = df_song[song_cols].values.tolist()[0]
song_data

['SONWXQJ12A8C134D94',
 'The Ballad Of Sleeping Beauty',
 'ARNF6401187FB57032',
 1994,
 305.162]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
artist_data_col =  ['artist_id'  , 'artist_name'  , 'artist_location'  , 'artist_latitude'  , 'artist_longitude']
artist_data = df_song[artist_data_col].values.tolist()[0]
artist_data

['ARNF6401187FB57032',
 'Sophie B. Hawkins',
 'New York, NY [Manhattan]',
 40.79086,
 -73.96644]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
log_files = 'data/log_data'

In [13]:
filepath_log = get_files(log_files)

In [14]:
df_log = pd.read_json(filepath_log[0], lines = True)
df_log_filtrado = df_log[df_log['page'] == 'NextSong']

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
t = pd.DataFrame(pd.to_datetime(df_log_filtrado['ts'], unit='ms'))
t.head()

,ts
0,2018-11-09 00:06:17.796
1,2018-11-09 00:09:46.796
2,2018-11-09 00:12:27.796
3,2018-11-09 00:14:52.796
4,2018-11-09 00:17:44.796


In [16]:
t = pd.to_datetime(df_log_filtrado['ts'], unit='ms')
time_data = list(( df_log_filtrado['ts'], t.dt.time, t.dt.hour, t.dt.day, t.dt.weekofyear, t.dt.month, t.dt.year, t.dt.weekday))
column_labels = ('ts','timestamp', 'hour', 'day', 'week', 'month', 'year','weekday')
time_df = pd.DataFrame(time_data, column_labels).T
(time_df[:3])

,ts,timestamp,hour,day,week,month,year,weekday
0,1541721977796,00:06:17.796000,0,9,45,11,2018,4
1,1541722186796,00:09:46.796000,0,9,45,11,2018,4
2,1541722347796,00:12:27.796000,0,9,45,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [18]:
column_labels = ['userId', 'firstName', 'lastName', 'gender','level']
user_df = df_log_filtrado[column_labels].drop_duplicates()
user_df[:5]


,userId,firstName,lastName,gender,level
0,42,Harper,Barrett,M,paid
34,49,Chloe,Cuevas,F,free
35,6,Cecilia,Owens,F,free
36,24,Layla,Griffin,F,paid
38,80,Tegan,Levine,F,paid


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [19]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

In [19]:

df_log_filtrado[:1]
#df[:1]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [20]:
for index, row in df_log_filtrado.iterrows():

    # get songid and artistid from song and artist tables
       
    cur.execute(song_select, (row.song, row.artist, row.length) )
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    print(results['song_id'])
        
     #songplay_id, start_time, user_id, song_id, session_id, location, user_agent
    #songplay_data = ( row['ts'],row['userId'], songid, row['sessionId'], row['location'], row['userAgent'] )
   
    #cur.execute(songplay_table_insert, songplay_data)
    #print(set(songplay_data))
    #cur.execute(songplay_table_insert, songplay_data)
    #conn.commit()

ValueError: not enough values to unpack (expected 2, got 1)

In [23]:
df_log_filtrado[:1]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Muse,Logged In,Harper,M,1,Barrett,209.50159,paid,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540685e+12,275,Supermassive Black Hole (Twilight Soundtrack V...,200,1541721977796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",42


In [21]:
#start_time, user_id, song_id, session_id, location, user_agent
#row['ts'],row['userId'], songid, row['sessionId'], row['location'], row['userAgent']

column_labels = ['ts', 'userId', 'sessionId', 'location','userAgent']
song_2_df = df_log_filtrado[column_labels].drop_duplicates()
song_2_df[:5]

,ts,userId,sessionId,location,userAgent
0,1541721977796,42,275,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
1,1541722186796,42,275,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
2,1541722347796,42,275,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
3,1541722492796,42,275,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."
4,1541722664796,42,275,"New York-Newark-Jersey City, NY-NJ-PA","""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK..."


In [22]:
for i, row in song_2_df.iterrows():
    
    cur.execute(song_select)
    results = cur.fetchone()
    
    songplay_data = ( row['ts'],row['userId'], results, row['sessionId'], row['location'], row['userAgent'] )
    
   # print(songplay_data)   
        
    cur.execute(songplay_table_insert, set(songplay_data))
    conn.commit()
    
    

(1541721977796, '42', ('SONWXQJ12A8C134D94',), 275, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(1541722186796, '42', ('SONWXQJ12A8C134D94',), 275, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(1541722347796, '42', ('SONWXQJ12A8C134D94',), 275, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(1541722492796, '42', ('SONWXQJ12A8C134D94',), 275, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(1541722664796, '42', ('SONWXQJ12A8C134D94',), 275, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.